# AE Experimentation

## Imports and Constants

In [1]:
import sys
import os

# Füge das übergeordnete Verzeichnis zu sys.path hinzu
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.insert(0, parent_dir)

In [41]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import seaborn as sns
import torch.nn.functional as F
from copy import deepcopy as dc

from utilities import split_data_into_sequences, train_test_split, load_sequential_time_series, reconstruct_sequential_data, Scaler, extract_features_and_targets_reg
from baseline_model.TimeSeriesDataset import TimeSeriesDataset

In [3]:
DATA_FOLDER = Path("../../data")
REAL_DATA_FOLDER = DATA_FOLDER / "real"
SYNTHETIC_DATA_FOLDER = DATA_FOLDER / "synthetic"
BENCHMARK = False

In [4]:
hyperparameters = {
    "seq_len": 12,
    "lr": 0.0001,
    "batch_size": 32,
    "hidden_size": 4,
    "num_layers": 1,
    "num_evaluation_runs": 10,
    "num_epochs": 200,
    "device": 'cuda' if torch.cuda.is_available() else 'cpu'
}

## Data

In [5]:
traffic_df = pd.read_csv(REAL_DATA_FOLDER / "metro_interstate_traffic_volume_label_encoded_no_categorical.csv")
traffic_df.shape

(28511, 5)

In [6]:
traffic_np = traffic_df.to_numpy()
traffic_np.shape

(28511, 5)

### Preprocessing

In [7]:
# Train test split
train, test = train_test_split(traffic_np, split_ratio=0.8)
test, val = train_test_split(test, split_ratio=0.5)

In [8]:
# scale data
scaler = Scaler(train)
train_scaled = scaler.scale_data(train)
val_scaled = scaler.scale_data(val)
test_scaled = scaler.scale_data(test)

In [9]:
# split data into sequences
train_seq_scaled = split_data_into_sequences(train_scaled, hyperparameters['seq_len'], shuffle_data=False)
val_seq_scaled = split_data_into_sequences(val_scaled, hyperparameters['seq_len'], shuffle_data=False)
test_seq_scaled = split_data_into_sequences(test_scaled, hyperparameters['seq_len'], shuffle_data=False)

Shape of the data after splitting into sequences: (22797, 12, 5)
Shape of the data after splitting into sequences: (2841, 12, 5)
Shape of the data after splitting into sequences: (2840, 12, 5)


In [10]:
# create datasets
# NOTE: The targets are not used in the training process, but are required by the PyTorch Dataset class
train_dataset = TimeSeriesDataset(train_seq_scaled, np.zeros((train_seq_scaled.shape[0], 1)))
val_dataset = TimeSeriesDataset(val_seq_scaled, np.zeros((val_seq_scaled.shape[0], 1)))
test_dataset = TimeSeriesDataset(test_seq_scaled, np.zeros((test_seq_scaled.shape[0], 1)))

In [11]:
# create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=hyperparameters['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False)

In [12]:
X_train, y = next(iter(train_loader))

## Autoencoder

In [90]:
class AutoencoderV1(nn.Module):
    def __init__(self, verbose=False):
        super().__init__()
        self.verbose = verbose
        # N, 12, 5
        
        self.relu = nn.ReLU(),
        self.sigmoid = nn.Sigmoid()  # Assuming the input data is in range [0, 1]
        
        self.conv1 = nn.Conv2d(1, 2, kernel_size=(2, 3), stride=2, padding=1)
        self.conv2 = nn.Conv2d(2, 4, kernel_size=(3, 2), stride=1, padding=0)
        self.conv3 = nn.Conv2d(4, 6, kernel_size=(5, 2), stride=1, padding=0) #n, 6, 1, 1
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(6, 4, kernel_size=(5, 2), stride=1, padding=0)
        self.deconv2 = nn.ConvTranspose2d(4, 2, kernel_size=(3, 2), stride=1, padding=0)
        self.deconv3 = nn.ConvTranspose2d(2, 1, kernel_size=(2, 3), stride=2, padding=(1, 1))
    
    def forward(self, x):
        x = x.unsqueeze(1)
        print(f'Input after unsqueeze: {x.shape}') if self.verbose else None

        x = F.relu(self.conv1(x))
        print(f'After conv1: {x.shape}') if self.verbose else None

        x = F.relu(self.conv2(x))
        print(f'After conv2: {x.shape}') if self.verbose else None

        x = F.relu(self.conv3(x))
        print(f'After conv3: {x.shape}') if self.verbose else None


        x = F.relu(self.deconv1(x))
        print(f'After conv_tran1: {x.shape}') if self.verbose else None

        x = F.relu(self.deconv2(x))
        print(f'After conv_tran2: {x.shape}') if self.verbose else None

        x = F.sigmoid(self.deconv3(x))
        print(f'After conv_tran3: {x.shape}') if self.verbose else None

        x = x.squeeze(1)
        return x

In [91]:
modelV1 = AutoencoderV1().to(hyperparameters['device'])

## Training

In [98]:
optimizer = torch.optim.Adam(params=modelV1.parameters(), lr=hyperparameters['lr'])
criterion = nn.MSELoss()

In [99]:
patience = 5
best_val_loss = np.inf
num_epochs_no_improvement = 0

for epoch in tqdm(range(hyperparameters['num_epochs'])):

    ### Training ###
    accumulated_train_loss = 0

    for _, (X_train, _) in enumerate(train_loader):

        modelV1.train()
        X_train = X_train.float().to(hyperparameters['device'])

        X_train_hat = modelV1(X_train)
        train_loss = criterion(X_train_hat, X_train)
        accumulated_train_loss += train_loss.item()

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()


    ### Validation ###
    accumulated_val_loss = 0
    modelV1.eval()
    with torch.inference_mode():

        for _, (X_val, _) in enumerate(val_loader):
            X_val = X_val.float().to(hyperparameters['device'])

            X_val_hat = modelV1(X_val)
            val_loss = criterion(X_val_hat, X_val)
            accumulated_val_loss += val_loss.item()


        # Check for early stopping
        if accumulated_val_loss < best_val_loss:
            best_val_loss = accumulated_val_loss
            num_epochs_no_improvement = 0

        else:
            print(f'INFO: Validation loss did not improve in epoch {epoch + 1}')
            num_epochs_no_improvement += 1


    ### Logging ###

    print(f'Epoch: {epoch} \n\b Train Loss: {accumulated_train_loss / len(train_loader)} \n\b Val Loss: {accumulated_val_loss / len(val_loader)}')
    print('*' * 50)

    if num_epochs_no_improvement >= patience:
        print(f'Early stopping at epoch {epoch}')
        break
        


  0%|          | 1/200 [00:04<15:57,  4.81s/it]

Epoch: 0, Train Loss: 0.1317879512362694


  2%|▏         | 4/200 [00:22<18:23,  5.63s/it]


KeyboardInterrupt: 